In [19]:
import tensorflow as tf
import numpy as np
import json
from collections import defaultdict

In [2]:
pairs = json.load(open('../data/fb-pairs.json'))

In [3]:
print pairs[0]

[[u'i have to get a filling\non the 13th'], u'Tbh it does help']


In [4]:
def flatten(lists):
    return [item for list in lists for item in list]


In [5]:
chars = 'abcdefghijklmnopqrstuvwxyz'
# chars += chars.upper() # lowercase everything
chars += ' 1234567890/:\'"_-()\n'
NONE = 0
START = 1
END = 2
chars = [NONE, START, END] + list(chars)


In [20]:
bucket_sizes = [64] # [8, 16, 32, 64]
buckets = [(in_size, out_size) for in_size in bucket_sizes for out_size in bucket_sizes]


In [21]:
def vectorize_pair(pair):
    def vectorize_str(str):
        return [chars.index(c) for c in str.lower() if c in chars]
    
    context = flatten([[START] + vectorize_str(str) for str in pair[0]])
    response = [START] + vectorize_str(pair[1]) + [END]
    
    def select_bucket(length):
        for bucket in bucket_sizes:
            if length <= bucket:
                return bucket
        return bucket_sizes[-1]
    
    def bucket_and_pad(seq, pad_on_front=False):
        bucket = select_bucket(len(seq))
        if pad_on_front:
            if len(seq) > bucket:
                seq = seq[-bucket:]
            return [NONE] * (bucket - len(seq)) + seq
        else:
            if len(seq) > bucket:
                seq = seq[:bucket]
            return seq + [NONE] * (bucket - len(seq))
    
    context = bucket_and_pad(context, True)
    response = bucket_and_pad(response, False)
    return (np.array(context), np.array(response))

def to_text(vec):
    def iterate_until_end(seq):
        for i in seq:
            if i == END:
                break
            yield i
    return ''.join([chars[i] for i in iterate_until_end(vec) if i > 2])

pair = [["MESSAGE", "CONTEXT!"], "RESPONSE goes here"]
ctx, resp = vectorize_pair(pairs[8])
# print ctx
print to_text(ctx)
print to_text(resp)

l doctor
no not the dentistohhh you went to your regular doctor
cheese


In [ ]:
vectorized_pairs = [vectorize_pair(msg) for msg in msgs]
